In [ ]:
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler

In [62]:
data_path = 'datasets/AAPL.csv'
data = pd.read_csv(data_path)
data

,Date,Adj Close,Close,High,Low,Open,Volume
0,2020-01-02,72.796013,75.087502,75.150002,73.797501,74.059998,135480400
1,2020-01-03,72.088287,74.357498,75.144997,74.125000,74.287498,146322800
2,2020-01-06,72.662712,74.949997,74.989998,73.187500,73.447502,118387200
3,2020-01-07,72.320961,74.597504,75.224998,74.370003,74.959999,108872000
4,2020-01-08,73.484360,75.797501,76.110001,74.290001,74.290001,132079200
...,...,...,...,...,...,...,...
1232,2024-11-22,229.869995,229.869995,230.720001,228.059998,228.059998,38168300
1233,2024-11-25,232.869995,232.869995,233.250000,229.740005,231.460007,90152800
1234,2024-11-26,235.059998,235.059998,235.570007,233.330002,233.330002,45986200
1235,2024-11-27,234.929993,234.929993,235.690002,233.809998,234.470001,33498400


In [63]:
data['Date'] = pd.to_datetime(data['Date']).astype('int64') // 10**9

In [64]:
X = data[['Date', 'Close', 'High', 'Low', 'Open', 'Volume']].values
y = data['Adj Close'].values

In [65]:
scaler = StandardScaler()

In [66]:
X_scaled = scaler.fit_transform(X)

In [67]:
def create_model():
    model = Sequential([
        Dense(16, activation='relu', input_shape=(6,)),
        Dense(8, activation='relu'),
        Dense(1)    
    ])
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mae', 'mse'])
    return model

In [68]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [69]:
results = []

In [70]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, random_state=42)

In [71]:
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=42)

In [72]:
for train_idx, val_idx in kf.split(X_scaled):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    model = create_model()
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=8, callbacks=[early_stopping], verbose=0)
    results.append(model.evaluate(X_val, y_val, verbose=0))

c:\MyFolder\Projects\University\ml-masters-sem3\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [73]:
print("Средние результаты по метрикам (MAE, MSE):", np.mean(results, axis=0))

Средние результаты по метрикам (MAE, MSE): [0.58292348 0.53194912 0.58292348]
